In [106]:
import numpy as np

In [107]:
NUM_USERS = 1000
ENTRANCE_FEE = 10 #USDT
TOKEN_RANGE = 50
TOKEN_SIZE = 6
WIN_AMOUNT = [0, 0, 20, 50, 100, 200, 800]
FIXED = [True, True, True, True, False, False]
REF_WIN_AMOUNT = [0, 0, 5, 10, 15, 20, 80]
LOTTERY_TOKENS = 1+np.random.choice(TOKEN_RANGE-1, TOKEN_SIZE)

In [108]:
def lottery_score(tok1, tok2):
    score = 0
    for i in range(TOKEN_SIZE):
        if tok1[i] in tok2:
            score += 1
    return score


def get_user(user_id):
    for u in users:
        if u.id == user_id:
            return u
    return None


class User:
    def __init__(self, user_id, referer_id):
        self.id = user_id
        self.referer = referer_id
        self. tokens = 1+np.random.choice(TOKEN_RANGE-1, TOKEN_SIZE)
        self.satisfaction = -ENTRANCE_FEE
        self.score = lottery_score(self.tokens, LOTTERY_TOKENS)
        self.free_play = True if np.random.rand() < TOKEN_SIZE/TOKEN_RANGE else False

In [109]:
# Generate random users
users = []
id = 1

users.append(User(id, 0))
id += 1
while len(users) < NUM_USERS:
    referer = users[np.random.randint(0, len(users))]
    users.append(User(id, referer.id))
    id += 1

bank = ENTRANCE_FEE * (NUM_USERS - sum([int(user.free_play) for user in users]))
print(bank)

8710


In [110]:
# Pick winners
referers_won = []
size_winners = (TOKEN_SIZE+1)*[0]
for user in users:
    size_winners[user.score] += 1
f = open("simulation.txt", "w")
for user in users:
    if WIN_AMOUNT[user.score] > 0:
        if FIXED[user.score]:
            user.satisfaction += WIN_AMOUNT[user.score]
            bank -= WIN_AMOUNT[user.score]
            f.write(f"User {user.id} won {WIN_AMOUNT[user.score]}$ => Bank: {bank}$\n")
            if user.referer != 0:
                if user.referer not in referers_won:
                    referers_won.append(user.referer)
                referer_user = users[user.referer-1] #get_user(user.referer)
                referer_user.satisfaction += REF_WIN_AMOUNT[user.score]
                bank -= REF_WIN_AMOUNT[user.score]
                f.write(f"User {referer_user.id} won {REF_WIN_AMOUNT[user.score]}$ by reference$ => Bank: {bank}$\n")
        else:
            user.satisfaction += WIN_AMOUNT[user.score] / size_winners[user.score]
            bank -= WIN_AMOUNT[user.score] / size_winners[user.score]
            f.write(f"User {user.id} won {WIN_AMOUNT[user.score] / size_winners[user.score]}$ => Bank: {bank}$\n")
            if user.referer != 0:
                if user.referer not in referers_won:
                    referers_won.append(user.referer)
                referer_user = users[user.referer-1] #get_user(user.referer)
                referer_user.satisfaction += REF_WIN_AMOUNT[user.score] / size_winners[user.score]
                bank -= REF_WIN_AMOUNT[user.score] / size_winners[user.score]
                f.write(f"User {referer_user.id} won {REF_WIN_AMOUNT[user.score] / size_winners[user.score]}$ by reference$ => Bank: {bank}$\n")
f.close()

In [114]:
# Show users data
unsatisfied = 0
satisfied = 0
f = open("users.txt", "w")
f.write(f"########## UNSATISFIED USERS ##########\n")
for user in users:
    if user.satisfaction < 0:
        unsatisfied += 1
        f.write(f"User {user.id} satisfaction: {user.satisfaction}\n")
f.write(f"Total number of unsatisfied users: {unsatisfied}\n")

f.write(f"########## SATISFIED USERS ##########\n")
for user in users:
    if user.satisfaction >= -5:
        satisfied += 1
        f.write(f"User {user.id} satisfaction: {user.satisfaction}\n")
f.write(f"Total number of satisfied users: {satisfied}\n")
f.close()